In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


sklearn version:  1.0.2
TF version:  2.8.0
GPU installed:  True
4 Physical GPUs, 4 Logical GPUs


2022-06-09 05:35:39.470470: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-09 05:35:41.997031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-06-09 05:35:41.998036: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14791 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-06-09 05:35:41.998983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

In [2]:
X_train=np.uint8(np.load('./X_train_aug.npy'))
y_train=np.uint8(np.load('./y_train_aug.npy'))
X_valid=np.uint8(np.load('./X_valid.npy'))
y_valid=np.uint8(np.load('./y_valid.npy'))

In [3]:
X_train.shape,X_valid.shape

((1074263, 32, 32, 1), (164015, 28, 28))

In [4]:
X_valid=tf.reshape(X_valid,[-1,28,28,1])
X_train=tf.reshape(X_train,[-1,32,32,1])
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))
X_train_resize=np.uint8(tf.image.resize(X_train, [32, 32]))
X_train_resize.shape,X_valid_resize.shape,X_train.dtype

2022-06-09 05:35:49.767211: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memory.


((1074263, 32, 32, 1), (164015, 32, 32, 1), tf.uint8)

In [5]:
X_train=keras.applications.mobilenet_v2.preprocess_input(np.array(X_train,np.float32))
X_valid_final=keras.applications.mobilenet_v2.preprocess_input(np.array(X_valid_resize,np.float32))

In [6]:
np.min(X_train)

-1.0

In [ ]:
#황성현 전민재 He_normal initializer
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        initializer = tf.keras.initializers.HeNormal()
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, kernel_initializer=initializer,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation,kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation,kernel_initializer=initializer),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,kernel_initializer=initializer,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation,kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation,kernel_initializer=initializer),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [7]:
#황성현 
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        initializer = tf.keras.initializers.HeNormal()
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, kernel_initializer=initializer,
                                padding="SAME"),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME",kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME",kernel_initializer=initializer),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,kernel_initializer=initializer,
                                padding="SAME"),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",kernel_initializer=initializer),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [8]:
    #황성현 전민재 IRB6512
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=512,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 32, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)     

In [10]:
#전민재
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)
    def on_epoch_end(self, batch, logs):
        print("lr: {}".format(self.model.optimizer.lr))

smv2sl_adam_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch40
lr =0.002

In [10]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime(f"smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch{epochs}%Y%m%d-%H%M%S")
print(logs)
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.002)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath=f"smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch{epochs}.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900


2022-06-08 09:29:00.404297: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-08 09:29:00.404350: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-08 09:29:00.404420: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 4 GPUs
2022-06-08 09:29:00.405233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-08 09:29:01.179426: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-08 09:29:01.179648: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-08 09:29:01.271363: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memor

Epoch 1/40


2022-06-08 09:29:08.696503: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204


 497/8393 [>.............................] - ETA: 3:02 - loss: 4.9271 - accuracy: 0.0420

...

 518/8393 [>.............................] - ETA: 3:16 - loss: 4.9033 - accuracy: 0.0433

2022-06-08 09:29:23.102770: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-08 09:29:23.104029: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-08 09:29:23.293156: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14785 callback api events and 14858 activity events. 
2022-06-08 09:29:23.537231: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-08 09:29:23.855002: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2022_06_08_09_29_23

2022-06-08 09:29:24.084162: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2

 525/8393 [>.............................] - ETA: 3:36 - loss: 4.8973 - accuracy: 0.0436

2022-06-08 09:29:24.357003: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2022_06_08_09_29_23

2022-06-08 09:29:24.364500: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2022_06_08_09_29_23/a58a2589147d.memory_profile.json.gz
2022-06-08 09:29:24.370768: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2022_06_08_09_29_23
Dumped tool data for xplane.pb to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-092900/plugins/profile/2022_06_08_09_29_23/a58a2589147d.xplane.pb
Dumped tool data for overview_p

8393/8393 [==============================] - 215s 25ms/step - loss: 1.9272 - accuracy: 0.4866 - val_loss: 0.5558 - val_accuracy: 0.8033
Epoch 2/40
8393/8393 [==============================] - 207s 25ms/step - loss: 0.5492 - accuracy: 0.8078 - val_loss: 0.4254 - val_accuracy: 0.8430
Epoch 3/40
8393/8393 [==============================] - 207s 25ms/step - loss: 0.4605 - accuracy: 0.8335 - val_loss: 0.4594 - val_accuracy: 0.8366
Epoch 4/40
8393/8393 [==============================] - 209s 25ms/step - loss: 0.4296 - accuracy: 0.8430 - val_loss: 0.4099 - val_accuracy: 0.8467
Epoch 5/40
8393/8393 [==============================] - 207s 25ms/step - loss: 0.4133 - accuracy: 0.8475 - val_loss: 0.4056 - val_accuracy: 0.8512
Epoch 6/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.4037 - accuracy: 0.8508 - val_loss: 0.4960 - val_accuracy: 0.8200
Epoch 7/40
8393/8393 [==============================] - 207s 25ms/step - loss: 0.3964 - accuracy: 0.8531 - val_loss: 0.7728 - val

maxlr 0.003 dropout 0.7

In [10]:
    #황성현 전민재 IRB6512, maxlr 0.003 dropout 0.7
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=512,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.7),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [11]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime(f"smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch{epochs}_noact%Y%m%d-%H%M%S")
print(logs)
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath=f"smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch{epochs}_noact.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558


2022-06-08 13:35:58.854450: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-08 13:35:58.854486: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-08 13:35:58.854531: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 4 GPUs
2022-06-08 13:35:58.855298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-08 13:35:59.659079: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-08 13:35:59.659340: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-08 13:35:59.747827: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memor

Epoch 1/40


2022-06-08 13:36:07.137433: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204


 499/8393 [>.............................] - ETA: 3:01 - loss: 5.5227 - accuracy: 0.0342

...

 518/8393 [>.............................] - ETA: 3:16 - loss: 5.4961 - accuracy: 0.0349

2022-06-08 13:36:21.614603: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-08 13:36:21.615946: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-08 13:36:21.801217: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14722 callback api events and 14795 activity events. 
2022-06-08 13:36:22.073949: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-08 13:36:22.386559: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profile/2022_06_08_13_36_22

2022-06-08 13:36:22.615295: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profi

 525/8393 [>.............................] - ETA: 3:36 - loss: 5.4865 - accuracy: 0.0352

2022-06-08 13:36:22.876807: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profile/2022_06_08_13_36_22

2022-06-08 13:36:22.883988: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profile/2022_06_08_13_36_22/a58a2589147d.memory_profile.json.gz
2022-06-08 13:36:22.889210: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profile/2022_06_08_13_36_22
Dumped tool data for xplane.pb to logs/smv2l_adam_best_lrsch002_onecycle_aug_dropout07_128_he_irb6_512_t4_epoch4020220608-133558/plugins/profile/2022_06_08_13_36_22/a58a2589147d.xplane.pb
Dumped tool data for ov

8393/8393 [==============================] - 214s 25ms/step - loss: 1.9257 - accuracy: 0.5047 - val_loss: 0.5293 - val_accuracy: 0.8153
Epoch 2/40
8393/8393 [==============================] - 216s 26ms/step - loss: 0.5492 - accuracy: 0.8102 - val_loss: 0.4349 - val_accuracy: 0.8416
Epoch 3/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.4715 - accuracy: 0.8327 - val_loss: 0.4145 - val_accuracy: 0.8477
Epoch 4/40
8393/8393 [==============================] - 206s 24ms/step - loss: 0.4435 - accuracy: 0.8411 - val_loss: 0.4285 - val_accuracy: 0.8424
Epoch 5/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.4287 - accuracy: 0.8451 - val_loss: 0.4331 - val_accuracy: 0.8379
Epoch 6/40
8393/8393 [==============================] - 207s 25ms/step - loss: 0.4198 - accuracy: 0.8481 - val_loss: 0.3875 - val_accuracy: 0.8585
Epoch 7/40
8393/8393 [==============================] - 208s 25ms/step - loss: 0.4136 - accuracy: 0.8501 - val_loss: 0.7183 - val

마지막 layer activation 제거

In [10]:
    #황성현 전민재 IRB61024, maxlr 0.003
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=1024,n=1,s=1,name="IRB6"),
        
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [11]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch40%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.003,start_rate=0.0003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug_dropout50_128_he_irb6_1024_t4_epoch40.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

2022-06-09 01:58:16.126621: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-09 01:58:16.126671: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-09 01:58:16.126739: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 4 GPUs
2022-06-09 01:58:16.130279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-09 01:58:16.897455: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-09 01:58:16.897675: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-09 01:58:16.990824: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memor

Epoch 1/40


2022-06-09 01:58:24.490574: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204


 499/8393 [>.............................] - ETA: 3:21 - loss: 4.8703 - accuracy: 0.0423

2022-06-09 01:58:38.763771: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-09 01:58:38.763828: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


 518/8393 [>.............................] - ETA: 3:36 - loss: 4.8510 - accuracy: 0.0433

2022-06-09 01:58:40.285731: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-09 01:58:40.287166: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-09 01:58:40.487802: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14680 callback api events and 14753 activity events. 
2022-06-09 01:58:40.736842: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-09 01:58:41.048911: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40

2022-06-09 01:58:41.278376: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40/a58a2589147d.tr

 525/8393 [>.............................] - ETA: 3:55 - loss: 4.8441 - accuracy: 0.0435

2022-06-09 01:58:41.549734: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40

2022-06-09 01:58:41.556964: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40/a58a2589147d.memory_profile.json.gz
2022-06-09 01:58:41.562301: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40
Dumped tool data for xplane.pb to logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch4020220609-015816/plugins/profile/2022_06_09_01_58_40/a58a2589147d.xplane.pb
Dumped tool data for overview_page.pb to logs/SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch

8393/8393 [==============================] - 214s 25ms/step - loss: 1.6326 - accuracy: 0.5595 - val_loss: 0.4990 - val_accuracy: 0.8211
Epoch 2/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.5140 - accuracy: 0.8169 - val_loss: 0.4271 - val_accuracy: 0.8436
Epoch 3/40
8393/8393 [==============================] - 205s 24ms/step - loss: 0.4528 - accuracy: 0.8355 - val_loss: 0.4111 - val_accuracy: 0.8503
Epoch 4/40
8393/8393 [==============================] - 205s 24ms/step - loss: 0.4264 - accuracy: 0.8435 - val_loss: 0.4050 - val_accuracy: 0.8489
Epoch 5/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.4111 - accuracy: 0.8482 - val_loss: 0.4003 - val_accuracy: 0.8518
Epoch 6/40
8393/8393 [==============================] - 206s 24ms/step - loss: 0.4015 - accuracy: 0.8514 - val_loss: 0.3955 - val_accuracy: 0.8548
Epoch 7/40
8393/8393 [==============================] - 206s 25ms/step - loss: 0.3956 - accuracy: 0.8535 - val_loss: 0.4521 - val

In [12]:
    #황성현 전민재 IRB61024, maxlr 0.003
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=32
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [ ]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch40%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.001,start_rate=0.0003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle001_aug_dropout50_128_he_irb6_1024_t4_epoch40.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

2022-06-09 05:37:19.292208: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-09 05:37:19.292237: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-09 05:37:20.120829: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-09 05:37:20.121055: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-09 05:37:20.226198: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memory.
2022-06-09 05:37:23.274603: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4400181248 exceeds 10% of free system memory.


Epoch 1/40
  499/33571 [..............................] - ETA: 12:21 - loss: 4.4551 - accuracy: 0.0889

2022-06-09 05:37:38.707555: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-09 05:37:38.707614: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


  518/33571 [..............................] - ETA: 13:20 - loss: 4.4274 - accuracy: 0.0906

2022-06-09 05:37:40.132170: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-09 05:37:40.133478: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-09 05:37:40.323830: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14050 callback api events and 14115 activity events. 
2022-06-09 05:37:40.563236: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-09 05:37:40.854569: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40

2022-06-09 05:37:41.065467: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40/a58a2589147d.

  526/33571 [..............................] - ETA: 14:38 - loss: 4.4146 - accuracy: 0.0918

2022-06-09 05:37:41.318784: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40

2022-06-09 05:37:41.326250: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40/a58a2589147d.memory_profile.json.gz
2022-06-09 05:37:41.331255: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40
Dumped tool data for xplane.pb to logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_epoch4020220609-053719/plugins/profile/2022_06_09_05_37_40/a58a2589147d.xplane.pb
Dumped tool data for overview_page.pb to logs/SMV2L_lrsch_onecycle001_dropout_128_he_irb6_1024_

33571/33571 [==============================] - 797s 24ms/step - loss: 0.8280 - accuracy: 0.7399 - val_loss: 0.4291 - val_accuracy: 0.8424
Epoch 2/40
33571/33571 [==============================] - 784s 23ms/step - loss: 0.4834 - accuracy: 0.8272 - val_loss: 0.4258 - val_accuracy: 0.8431
Epoch 3/40
33571/33571 [==============================] - 787s 23ms/step - loss: 0.4458 - accuracy: 0.8382 - val_loss: 0.3848 - val_accuracy: 0.8572
Epoch 4/40
33571/33571 [==============================] - 787s 23ms/step - loss: 0.4255 - accuracy: 0.8442 - val_loss: 0.3721 - val_accuracy: 0.8620
Epoch 5/40
33571/33571 [==============================] - 788s 23ms/step - loss: 0.4135 - accuracy: 0.8481 - val_loss: 0.3672 - val_accuracy: 0.8628
Epoch 6/40
33571/33571 [==============================] - 786s 23ms/step - loss: 0.4045 - accuracy: 0.8508 - val_loss: 0.3615 - val_accuracy: 0.8648
Epoch 7/40
33571/33571 [==============================] - 789s 24ms/step - loss: 0.3981 - accuracy: 0.8530 - val_loss

In [ ]:
smv2_l.save('./smv2l_adam_lrsch_onecycle001_aug_dropout_128_he_irb6_512_noact.h5')